In [1]:
!pwd

/users/avanti/av_scripts/adhoc/DeepSEAtoKeras


In [2]:
!conda list

# packages in environment at /users/avanti/anaconda3:
#
# Name                    Version                   Build  Channel
_ipyw_jlab_nb_ext_conf    0.1.0                    py37_0  
_libgcc_mutex             0.1                        main  
absl-py                   0.7.1                    pypi_0    pypi
abstention                0.1.2.1                  pypi_0    pypi
alabaster                 0.7.12                   py37_0  
anaconda                  2019.03                  py37_0  
anaconda-client           1.7.2                    py37_0  
anaconda-navigator        1.9.7                    py37_0  
anaconda-project          0.8.2                    py37_0  
asn1crypto                0.24.0                   py37_0  
astor                     0.8.0                    pypi_0    pypi
astroid                   2.2.5                    py37_0  
astropy                   3.1.2            py37h7b6447c_0  
atomicwrites              1.3.0                    py37_1  
attrs              

In [3]:
from __future__ import print_function, absolute_import
import torch
from torch import nn

class LambdaBase(nn.Sequential):
    def __init__(self, fn, *args):
        super(LambdaBase, self).__init__(*args)
        self.lambda_func = fn

    def forward_prepare(self, input):
        output = []
        for module in self._modules.values():
            output.append(module(input))
        return output if output else input

class Lambda(LambdaBase):
    def forward(self, input):
        return self.lambda_func(self.forward_prepare(input))

class Beluga(nn.Module):
    def __init__(self):
        super(Beluga, self).__init__()
        self.model = nn.Sequential(
            nn.Sequential(
                nn.Conv2d(4,320,(1, 8)),
                nn.ReLU(),
                nn.Conv2d(320,320,(1, 8)),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.MaxPool2d((1, 4),(1, 4)),
                nn.Conv2d(320,480,(1, 8)),
                nn.ReLU(),
                nn.Conv2d(480,480,(1, 8)),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.MaxPool2d((1, 4),(1, 4)),
                nn.Conv2d(480,640,(1, 8)),
                nn.ReLU(),
                nn.Conv2d(640,640,(1, 8)),
                nn.ReLU(),
            ),
            nn.Sequential(
                nn.Dropout(0.5),
                Lambda(lambda x: x.view(x.size(0),-1)),
                nn.Sequential(Lambda(lambda x: x.view(1,-1) if 1==len(x.size()) else x ),nn.Linear(67840,2003)),
                nn.ReLU(),
                nn.Sequential(Lambda(lambda x: x.view(1,-1) if 1==len(x.size()) else x ),nn.Linear(2003,2002)),
            ),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.model(x)

pytorch_model = Beluga()
#srv scratch on nandi
pytorch_model.load_state_dict(torch.load('/srv/scratch/avanti/ExPecto/resources/deepsea.beluga.pth'))



<All keys matched successfully>

In [4]:
pytorch_model

Beluga(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(4, 320, kernel_size=(1, 8), stride=(1, 1))
      (1): ReLU()
      (2): Conv2d(320, 320, kernel_size=(1, 8), stride=(1, 1))
      (3): ReLU()
      (4): Dropout(p=0.2, inplace=False)
      (5): MaxPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(320, 480, kernel_size=(1, 8), stride=(1, 1))
      (7): ReLU()
      (8): Conv2d(480, 480, kernel_size=(1, 8), stride=(1, 1))
      (9): ReLU()
      (10): Dropout(p=0.2, inplace=False)
      (11): MaxPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0, dilation=1, ceil_mode=False)
      (12): Conv2d(480, 640, kernel_size=(1, 8), stride=(1, 1))
      (13): ReLU()
      (14): Conv2d(640, 640, kernel_size=(1, 8), stride=(1, 1))
      (15): ReLU()
    )
    (1): Sequential(
      (0): Dropout(p=0.5, inplace=False)
      (1): Lambda()
      (2): Sequential(
        (0): Lambda()
        (1): Linear(in_features=67840, out_features

In [5]:
#creating a keras model with the right architecture
import keras
import numpy as np

keras_model = keras.models.Sequential()
keras_model.add(keras.layers.convolutional.Conv1D(
            filters=320, kernel_size=8,
            strides=1, padding="valid",
            batch_input_shape=(None, 2000, 4)))
keras_model.add(keras.layers.Activation("relu"))
keras_model.add(keras.layers.convolutional.Conv1D(
            filters=320, kernel_size=8,
            strides=1, padding="valid",
            batch_input_shape=(None, 1000, 4)))
keras_model.add(keras.layers.Activation("relu"))
keras_model.add(keras.layers.core.Dropout(0.2))
keras_model.add(keras.layers.pooling.MaxPooling1D(
           pool_size=4, strides=4, padding="valid"))
keras_model.add(keras.layers.convolutional.Conv1D(
            filters=480, kernel_size=8,
            strides=1, padding="valid"))
keras_model.add(keras.layers.Activation("relu"))
keras_model.add(keras.layers.convolutional.Conv1D(
            filters=480, kernel_size=8,
            strides=1, padding="valid"))
keras_model.add(keras.layers.Activation("relu"))
keras_model.add(keras.layers.core.Dropout(0.2))
keras_model.add(keras.layers.pooling.MaxPooling1D(
           pool_size=4, strides=4, padding="valid"))
keras_model.add(keras.layers.convolutional.Conv1D(
            filters=640, kernel_size=8,
            strides=1, padding="valid"))
keras_model.add(keras.layers.Activation("relu"))
keras_model.add(keras.layers.convolutional.Conv1D(
            filters=640, kernel_size=8,
            strides=1, padding="valid"))
keras_model.add(keras.layers.Activation("relu"))
keras_model.add(keras.layers.core.Dropout(0.5))
keras_model.add(keras.layers.core.Permute((2,1)))
keras_model.add(keras.layers.core.Flatten())
keras_model.add(keras.layers.core.Dense(2003))
keras_model.add(keras.layers.Activation("relu"))
keras_model.add(keras.layers.core.Dense(2002))
keras_model.add(keras.layers.Activation("sigmoid"))
keras_model.build()

Using TensorFlow backend.
W0821 21:36:46.974293 139846186448640 deprecation_wrapper.py:119] From /users/avanti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0821 21:36:47.006508 139846186448640 deprecation_wrapper.py:119] From /users/avanti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0821 21:36:47.009881 139846186448640 deprecation_wrapper.py:119] From /users/avanti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0821 21:36:47.080435 139846186448640 deprecation_wrapper.py:119] From /users/avanti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Ple

In [6]:
#Port the DeepSEA weights over

#need to rearrange order of input bases from AGCT to ACGT, hence [0,2,1,3]
#also conversion from conv2d to conv1d and a different channel ordering
conv1_weight = (list(list(list(pytorch_model.children())[0].children())[0].children())[0]
                 .weight.detach().cpu().numpy().squeeze()[:,[0,2,1,3]]).transpose((2,1,0))
conv1_bias = (list(list(list(pytorch_model.children())[0].children())[0].children())[0]
              .bias.detach().cpu().numpy())
print("conv1 weights shape", conv1_weight.shape)
print("conv1 bias shape", conv1_bias.shape)
keras_model.layers[0].set_weights([conv1_weight, conv1_bias])


conv2_weight = (list(list(list(pytorch_model.children())[0].children())[0].children())[2]
                 .weight.detach().cpu().numpy().squeeze()).transpose((2,1,0))
conv2_bias = (list(list(list(pytorch_model.children())[0].children())[0].children())[2]
              .bias.detach().cpu().numpy())
print("conv2 weights shape", conv2_weight.shape)
print("conv2 bias shape", conv2_bias.shape)
keras_model.layers[2].set_weights([conv2_weight, conv2_bias])


conv3_weight = (list(list(list(pytorch_model.children())[0].children())[0].children())[6]
                .weight.detach().cpu().numpy().squeeze().transpose((2,1,0)))
conv3_bias = (list(list(list(pytorch_model.children())[0].children())[0].children())[6]
              .bias.detach().cpu().numpy())
print("conv3 weights shape", conv3_weight.shape)
print("conv3 bias shape", conv3_bias.shape)
keras_model.layers[6].set_weights([conv3_weight, conv3_bias])


conv4_weight = (list(list(list(pytorch_model.children())[0].children())[0].children())[8]
                .weight.detach().cpu().numpy().squeeze().transpose((2,1,0)))
conv4_bias = (list(list(list(pytorch_model.children())[0].children())[0].children())[8]
              .bias.detach().cpu().numpy())
print("conv4 weights shape", conv4_weight.shape)
print("conv4 bias shape", conv4_bias.shape)
keras_model.layers[8].set_weights([conv4_weight, conv4_bias])


conv5_weight = (list(list(list(pytorch_model.children())[0].children())[0].children())[12]
                .weight.detach().cpu().numpy().squeeze().transpose((2,1,0)))
conv5_bias = (list(list(list(pytorch_model.children())[0].children())[0].children())[12]
              .bias.detach().cpu().numpy())
print("conv5 weights shape", conv5_weight.shape)
print("conv5 bias shape", conv5_bias.shape)
keras_model.layers[12].set_weights([conv5_weight, conv5_bias])


conv6_weight = (list(list(list(pytorch_model.children())[0].children())[0].children())[14]
                .weight.detach().cpu().numpy().squeeze().transpose((2,1,0)))
conv6_bias = (list(list(list(pytorch_model.children())[0].children())[0].children())[14]
                .bias.detach().cpu().numpy())
print("conv6 weights shape", conv5_weight.shape)
print("conv6 bias shape", conv5_bias.shape)
keras_model.layers[14].set_weights([conv6_weight, conv6_bias])


dense1_weight = (list(list(list(list(pytorch_model.children())[0].children())[1].children())[2].children())[1]
                 .weight.detach().cpu().numpy().squeeze().transpose((1,0)))
dense1_bias = (list(list(list(list(pytorch_model.children())[0].children())[1].children())[2].children())[1]
                 .bias.detach().cpu().numpy())
print("dense1 weights shape", dense1_weight.shape)
print("dense1 bias shape", dense1_bias.shape)
keras_model.layers[19].set_weights([dense1_weight, dense1_bias])


dense2_weight = (list(list(list(list(pytorch_model.children())[0].children())[1].children())[4].children())[1]
                 .weight.detach().cpu().numpy().squeeze().transpose((1,0)))
dense2_bias = (list(list(list(list(pytorch_model.children())[0].children())[1].children())[4].children())[1]
                 .bias.detach().cpu().numpy())
print("dense2 weights shape", dense2_weight.shape)
print("dense2 bias shape", dense2_bias.shape)
keras_model.layers[21].set_weights([dense2_weight, dense2_bias])

W0821 21:36:47.326216 139846186448640 deprecation_wrapper.py:119] From /users/avanti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



conv1 weights shape (8, 4, 320)
conv1 bias shape (320,)
conv2 weights shape (8, 320, 320)
conv2 bias shape (320,)
conv3 weights shape (8, 320, 480)
conv3 bias shape (480,)
conv4 weights shape (8, 480, 480)
conv4 bias shape (480,)
conv5 weights shape (8, 480, 640)
conv5 bias shape (640,)
conv6 weights shape (8, 480, 640)
conv6 bias shape (640,)
dense1 weights shape (67840, 2003)
dense1 bias shape (2003,)
dense2 weights shape (2003, 2002)
dense2 bias shape (2002,)


In [7]:
keras_model.save("/srv/scratch/avanti/ExPecto/resources/deepseabeluga_keras.h5")

In [8]:
def get_pytorch_preds(list_of_layers, the_input):
    for layer in list_of_layers:
        the_input = layer.eval().forward(the_input)
    return the_input

np.random.seed(1)
dummy_data = np.random.random((10,4,1,2000))
#one-hot encode it
dummy_data = (dummy_data==np.max(dummy_data,axis=1)[:,None,:,:]).astype("float32")
pytorch_predictions = get_pytorch_preds(
    list_of_layers=[pytorch_model],
    the_input=torch.from_numpy(dummy_data)).detach().cpu().numpy()


In [9]:
from keras.models import Model
#conv_model = Model(inputs=keras_model.inputs, outputs=keras_model.layers[15].output)
keras_predictions = keras_model.predict(dummy_data.squeeze().transpose(0,2,1)[:,:,[0,2,1,3]])
max_diff = np.max(np.abs(pytorch_predictions-keras_predictions))
print(max_diff)
assert max_diff < 10**-5


8.046627e-07
